<img src="https://www.telecom-paristech.fr/fileadmin/documents/images/logos/TelecomParisTech/TELECOM_PARISTECH_IMT_grise_RVB.png
"  WIDTH=100 HEIGHT=100> <img src="./smovengo.jpg" width=220 height=100>



# Velib à 5 stations
## SPEIT 2018
### Simulation en Python


----------------------------

## Initialisation

In [1]:
from numpy import *
import numpy as np

+ Matrice de routage

In [2]:
R=np.array([[0,0.2,0.3,0.2,0.3],[0.2,0.,0.3,0.2,0.3],[0.2,0.25,0.,0.25,0.3],[0.15,0.2,0.3,0.,0.35],[0.2,0.25,0.35,0.2,0.]])
print(R)

[[0.   0.2  0.3  0.2  0.3 ]
 [0.2  0.   0.3  0.2  0.3 ]
 [0.2  0.25 0.   0.25 0.3 ]
 [0.15 0.2  0.3  0.   0.35]
 [0.2  0.25 0.35 0.2  0.  ]]


+ Temps de traversées en minutes

In [3]:
T=np.array([[0,3.,5.,7.,7.],[2.,0,2.,5.,5.],[4.,2.,0,3.,3.],[8.,6.,4.,0,2.],[7.,7.,5.,2.,0]])
print(T)

[[0. 3. 5. 7. 7.]
 [2. 0. 2. 5. 5.]
 [4. 2. 0. 3. 3.]
 [8. 6. 4. 0. 2.]
 [7. 7. 5. 2. 0.]]


In [4]:
Mu = np.divide(1, T, where=T!=0)
print(Mu)

[[0.         0.33333333 0.2        0.14285714 0.14285714]
 [0.5        0.         0.5        0.2        0.2       ]
 [0.25       0.5        0.         0.33333333 0.33333333]
 [0.125      0.16666667 0.25       0.         0.5       ]
 [0.14285714 0.14285714 0.2        0.5        0.        ]]


+ Taux d'arrivées des clients dans chaque station en clients/minutes

In [5]:
lambda_=np.array([2.8,3.7,5.5,3.5,4.6])/60.
print(lambda_)

[0.04666667 0.06166667 0.09166667 0.05833333 0.07666667]


## Représentation de l'espace d'états
Il y a les vélos disponibles en stations que l'on  noté $N_{ii}$ et les vélos en cours de transition de la station $i$ vers la station $j$ dont on a noté le nombre par $N_{ij}$.

** La commande np.random.choice **

Pour se simplifier la vie, on doit, plusieurs fois, faire appel à la commande 

```python
np.random.choice(valeurs,p=proba)
```
qui retourne un tirage dans _valeurs_ selon le vecteur de probabillités _proba_
Ainsi
```python
np.random.choice(3,p=[0.2,0.5,0.3])
```
renvoie $0$ avec probabilité $0.2$, renvoie $1$ avec probabilité $0.5$, etc.

In [6]:
for i in np.arange(5):
    print(np.random.choice(5,p=R[0]))

4
3
2
4
1


Il en découle qu'il est plus simple de représenter un état sous forme d'un vecteur de longueur 25 $E=(E_i,0\le i \le 24)$, version dépliée de la matrice 5x5 $N=(N_{i,j}, 0\le i,j\le 4)$. Ce qui veut dire que
\begin{align*}
N_{ii} & \Longleftrightarrow E[6*i]\\
N_{ij} & \Longleftrightarrow E[5*i+j]
\end{align*}

### Fonctions
On fera trois fonctions

```python
def transition(E):
    ...
    return Tpos
```
Cette fonction _transition_ prend un état E et renvoie le vecteur des transitions possibles Tpos.
Une façon de représenter les transitions possibles est la suivante :
+ Tpos[6*i] vaut le taux de départ de la station i s'il y reste au moins un vélo
+ Tpos[5*i+j] vaut le taux de transition de $N$-> $N_{ij}-1,\ N_j+1$

```python
def saut(E):
    ...
    return tempsSejour,caseADecrementer,caseAIncrementer
```
Cette fonction _saut_ fait appel à la précédente et retourne, le temps de séjour dans l'état E, l'indice de l'élément de E qui va être diminué de 1 (noté *depart*) et l'indice de E qui va être augmenté de 1 (noté *arrivee*)

```python
def trajectoire(Einitial,horizon)
    ...
    return Tvide/horizon
```

Cette fonction simule une trajectoire partant de l'état *Einitial*, jusqu'au temps *horizon*.
Elle retourne le vecteur *Tvide* de longueur 5 qui contient le pourcentage du temps pendant lequel chaque station est vide.

In [61]:
def transition(etat):
    notEmpty = [[1,0][etat[i]==0] for i in range(25)]
    Tpos_arrive = np.multiply(etat,Mu.flatten())
    Tpos_depart = np.multiply(np.diag(lambda_).flatten(),notEmpty)

    Tpos = Tpos_arrive + Tpos_depart
    
    return Tpos

def saut(etat):
    T = transition(etat)
    tau = np.sum(T)
    choix = np.random.choice(25,p=(T/tau))
    depart, arrive = int(choix / 5), int(choix %5)
    caseADiminuer = choix
    
    if depart != arrive :
        caseAAugmenter = 6 * arrive
    else :
        prochaineStat = np.random.choice(5,p=R[depart])
        caseAAugmenter = 5 * depart + prochaineStat
    return -log(np.random.rand())/tau,caseADiminuer,caseAAugmenter

def trajectoire(etat,horizon):
    t = 0
    Tvide = np.zeros(5)
    Evide = np.diag(np.array([[0,1][etat[i]==0] for i in range(25)]).reshape(5,5))
   
    
    while t <= horizon:
        duree, caseADiminuer, caseAAugmenter = saut(etat)
        etat[caseADiminuer] = etat[caseADiminuer] - 1
        etat[caseAAugmenter] = etat[caseAAugmenter] + 1
        Tvide = Tvide + min(duree, horizon-t) * Evide
        Evide = np.diag(np.array([[0,1][etat[i]==0] for i in range(25)]).reshape(5,5))
        t = t + duree

    return Tvide/horizon





## Simulation 

+ *Einitial* est le vecteur représentant l'état initial
+ *Nbiter* le nombre de réalisations
+ *Horizon* l'horizon temporel de chaque réalisation

In [58]:
def simulation(Einitial,Nbiter,Horizon):
    stationsVides=np.empty(5)
    for k in np.arange(Nbiter):
        stationsVides=np.vstack((stationsVides,trajectoire(Einitial,Horizon)))
    print("Pourcentage du temps où les stations sont vides\n")
    for i in np.arange(5):
        m=1-np.sum(stationsVides[:,i])/Nbiter
        print('Station '+str(i)+' : {:.3f}'.format(m))
        v=1.96*np.std(stationsVides[:,i])/np.sqrt(Nbiter)
        print('Intervalle de confiance : [{:.3f}'.format(m-v)+', {:.3f}'.format(m+v)+']')
        print("-------------\n")


## Calcul de la probabilité stationnaire pour 1 vélo

Pour valider votre code, rien de tel que de le vérifier sur une situation où l'on peut calculer les quantités intéressantes de façon théorique. Pour $1$ vélo, on doit avoir
\begin{equation*}
\sum_{i=0}^4 \sum_{j=0}^4 N_{i,j}=1 
\end{equation*}
donc l'espace d'états se résume à l'ensemble des vecteurs à 25 composantes où une seule composante est non nulle et vaut $1$, il y a donc 25 états !

Vous devez
1. construire la matrice 25x25 de transition du système Vélib dans ce cas, noté $P$

2. remplacer la dernière colonne de cette matrice par une colonne de 1

3. construire le vecteur de taille 25 : $v=(0,\cdots,0,1)$ 
4. résoudre $P^t x=v$
5. exprimer le pourcentage du temps où la station $i$ est vide en fonction de $x$
6. comparer ces valeurs théoriques et exactes à celles données par votre simulation. Tant qu'elles ne coïncident pas, i.e. tant que les valeurs théoriques ne sont pas dans l'intervalle de confiance, c'est que vous avez un problème... 


_*Pour indication, vous devez trouver comme probabilité stationnaire*_

```python
array([ 0.17546728,  0.15550435,  0.13452462,  0.15566816,  0.15927423])
```


In [63]:
# Calcul des probabilités stationnaires pour 1 vélo
Einitial=np.zeros(25)
Einitial[0]=1
simulation(Einitial,1,10000.)

# Test de l'accord des valeurs avec Einitial=[1,0,0,...]


[8124.72802989 8350.8301808  8824.92437167 8588.56209512 8422.55494438]
Pourcentage du temps où les stations sont vides

Station 0 : 0.188
Intervalle de confiance : [-0.609, 0.984]
-------------

Station 1 : 0.165
Intervalle de confiance : [-0.653, 0.983]
-------------

Station 2 : 0.118
Intervalle de confiance : [-0.747, 0.982]
-------------

Station 3 : 0.141
Intervalle de confiance : [-0.701, 0.983]
-------------

Station 4 : 0.158
Intervalle de confiance : [-0.668, 0.983]
-------------



/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  
/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


## Calcul pour les données réelles
Maintenant, il ne vous reste plus qu'à faire tourner votre programme avec 100 vélos. 
### Expliquer pourquoi on peut choisir un état de départ quelconque.

In [62]:
Einitial=np.zeros(25)
for i in np.arange(100):
    Einitial[np.random.randint(25)]+=1
print(Einitial)
simulation(Einitial,1,10000.)

[3. 5. 4. 3. 4. 2. 8. 3. 4. 2. 1. 6. 7. 5. 2. 2. 6. 4. 4. 4. 4. 4. 6. 2.
 5.]


/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  
/Users/Nina/Documents/DL/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


[   0.            0.         1688.19653017  161.30409904 1047.37789749]
Pourcentage du temps où les stations sont vides

Station 0 : 1.000
Intervalle de confiance : [1.000, 1.000]
-------------

Station 1 : 1.000
Intervalle de confiance : [1.000, 1.000]
-------------

Station 2 : 0.831
Intervalle de confiance : [0.666, 0.997]
-------------

Station 3 : 0.984
Intervalle de confiance : [0.968, 1.000]
-------------

Station 4 : 0.895
Intervalle de confiance : [0.793, 0.998]
-------------

